# Example of `InferenceData` schema in PyMC3
The description of the `InferenceData` structure can be found in [GitHub](https://github.com/arviz-devs/arviz/blob/master/schema.md).

In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import numpy as np
import xarray
# xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,25200,12.0
Erika,27500,18.0


In [3]:
with pm.Model() as model:
    time_since_joined = pm.Data("time_since_joined", time)
    
    sigma = pm.HalfNormal('sigma', sd=30)
    b0 = pm.Normal("b0", mu=0, sd=10)
    b1 = pm.Normal("b1", mu=0, sd=10)
    
    pm.Normal("slack_comments", mu=b0 + b1 * time_since_joined, sigma=sigma, observed=slack_comments)
    
    trace = pm.sample(200, chains=4)
    posterior_predictive = pm.sample_posterior_predictive(trace)
    prior = pm.sample_prior_predictive(150)

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b1, b0, sigma]
Sampling 4 chains, 0 divergences: 100%|██████████| 2800/2800 [00:03<00:00, 834.18draws/s] 
The acceptance probability does not match the target. It is 0.9387873794290098, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8838650901566877, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8858350677206992, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9086067598818758, but should be close to 0.8. Try to increase the number of tuning steps.
100%|██████████| 800/800 [00:00<00:00, 1336.19it/s]


In [4]:
idata_pymc3 = az.from_pymc3(
    trace,
    prior=prior,
    posterior_predictive=posterior_predictive,
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "log_likelihood": ["developer"],
        "time_since_joined": ["developer"],
    }
)

In [5]:
idata_pymc3

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data
	> constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [6]:
idata_pymc3.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 200)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 192 193 194 195 196 197 198 199
Data variables:
    b0       (chain, draw) float64 -3.075 18.94 17.4 15.94 ... -2.506 25.35 27.1
    b1       (chain, draw) float64 95.89 64.52 64.21 67.26 ... 92.3 72.77 71.42
    sigma    (chain, draw) float64 1.116e+03 1.14e+03 ... 1.113e+03 1.117e+03
Attributes:
    created_at:                 2019-11-14T23:26:56.083499
    inference_library:          pymc3
    inference_library_version:  3.7

In [7]:
idata_pymc3.sample_stats

<xarray.Dataset>
Dimensions:           (chain: 4, developer: 5, draw: 200)
Coordinates:
  * chain             (chain) int64 0 1 2 3
  * draw              (draw) int64 0 1 2 3 4 5 6 ... 193 194 195 196 197 198 199
  * developer         (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    energy            (chain, draw) float64 1.468e+03 1.468e+03 ... 1.466e+03
    tune              (chain, draw) bool True False False ... False False False
    step_size         (chain, draw) float64 0.8691 0.8691 ... 0.9082 0.9082
    step_size_bar     (chain, draw) float64 0.6369 0.6369 ... 0.9218 0.9218
    tree_size         (chain, draw) float64 3.0 7.0 1.0 3.0 ... 3.0 7.0 3.0 1.0
    diverging         (chain, draw) bool False False False ... False False False
    depth             (chain, draw) int64 2 3 1 2 2 2 2 3 3 ... 2 2 2 2 2 3 2 1
    energy_error      (chain, draw) float64 0.06014 -0.00658 ... 0.06489
    lp                (chain, draw) float64 -1.465e+03 -1.466e+03 ... -1.465e+03
    mean_tree_accept  (chain, draw) float64 0.6886 0.9033 1.0 ... 0.9597 0.9372
    max_energy_error  (chain, draw) float64 0.7113 0.2662 ... -0.2534 0.06489
    log_likelihood    (chain, draw, developer) float64 -28.0 -44.36 ... -282.5
Attributes:
    created_at:                 2019-11-14T23:26:56.170685
    inference_library:          pymc3
    inference_library_version:  3.7

In [8]:
idata_pymc3.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 200)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 87.73 ... -1.204e+03
Attributes:
    created_at:                 2019-11-14T23:26:56.173735
    inference_library:          pymc3
    inference_library_version:  3.7

In [9]:
idata_pymc3.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) float64 7.5e+03 1.01e+04 ... 2.52e+04 2.75e+04
Attributes:
    created_at:                 2019-11-14T23:26:56.177099
    inference_library:          pymc3
    inference_library_version:  3.7

In [10]:
idata_pymc3.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2019-11-14T23:26:56.177781
    inference_library:          pymc3
    inference_library_version:  3.7

In [11]:
idata_pymc3.prior

<xarray.Dataset>
Dimensions:         (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain           (chain) int64 0
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    b1              (chain, draw) float64 0.7413 -5.262 -13.12 ... 14.67 -17.09
    sigma           (chain, draw) float64 9.342 2.451 34.68 ... 3.495 13.73
    sigma_log__     (chain, draw) float64 2.235 0.8964 3.546 ... 1.251 2.619
    b0              (chain, draw) float64 -12.77 -0.09907 6.602 ... 1.464 -11.77
    slack_comments  (chain, draw, developer) float64 -22.21 -6.414 ... -318.9
Attributes:
    created_at:                 2019-11-14T23:26:56.175309
    inference_library:          pymc3
    inference_library_version:  3.7